This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [224]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, regularizers
import numpy as np
import shutil
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [225]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.  
train_dir = "datasets/cats_vs_dogs_split/train"  # e.g. './muffin-vs-chihuahua/train'
test_dir = "datasets/cats_vs_dogs_split/test"    # e.g. './muffin-vs-chihuahua/test'

In [ ]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [227]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    shuffle=True
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle=True
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)
print(train_generator.class_indices)
x_batch, y_batch = next(train_generator)
print("Batch labels:", y_batch[:10])


Found 556 images belonging to 2 classes.
Found 138 images belonging to 2 classes.
Found 151 images belonging to 2 classes.
{'cats': 0, 'dogs': 1}
Batch labels: [1. 0. 0. 0. 0. 0. 0. 0. 1. 1.]


In [228]:
# SIMPLE CNN MODEL ARCHITECTURE

# Some modifications are applied
initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Applied dropout layers to reduce overfitting
#Improvements Made, Dropout(Reduce Overfitting), L2 Regularization
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [229]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [230]:
# TRAINING THE CNN
# Increased to 15 epochs for better training
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator
)

Epoch 1/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 7s 113ms/step - accuracy: 0.5090 - loss: 0.0000e+00 - val_accuracy: 0.5145 - val_loss: 0.0000e+00
Epoch 2/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.5126 - loss: 0.0000e+00 - val_accuracy: 0.5145 - val_loss: 0.0000e+00
Epoch 3/15
 9/33 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.5247 - loss: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.4834 - loss: 0.0000e+00
Test Accuracy: 0.4834437072277069


In [ ]:
# SAVE THE MODEL
model.save('exercise_6_custom_ledesma.h5')

In [ ]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='exercise_6_custom_ledesma.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    pred = model.predict(img_array)[0,0]
    if train_generator.class_indices['dogs'] == 1:
        label = "Dog" if pred >= 0.5 else "Cat"
        confidence = pred if pred >= 0.5 else 1 - pred
    else:  # dogs=0, cats=1
        label = "Dog" if pred < 0.5 else "Cat"
        confidence = 1 - pred if pred < 0.5 else pred
    print(f"Prediction: {label} (confidence: {pred:.2f})")


In [ ]:
# Example usage:
predict_image("datasets/cats_vs_dogs_split/test/cats/cat.4002.jpg")
predict_image("datasets/cats_vs_dogs_split/test/dogs/dog.4010.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step


Prediction: Cat (confidence: 0.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
Prediction: Cat (confidence: 0.00)
